# PTST Idealization using axi-symmetric 2D interface layer

## Purpose of the notebook

Verify the axi-symmetric model in an elastic state to ensure that the confinement is
correctly reproduced. This notebook serves as a basis for 3D degradation within
a ligament/bond zone in axi-symmetric models. Examples are
 - PTST
 - prestressing
 - confined bond configuration
 - effect of splitting cracks
 - restrained splitting cracks in fatigue tests

## Implementation remarks

- The notebook now uses `BCSliceI` - to ensure the same order of nodes along an 
interfaces
- Compression is applied using individual `BCDof` instances without integration 
  over the boundary. This should be changed to BCSliceE - which can use the 
  finite element shape functions to integrate over the element domain

In [91]:
%matplotlib widget
import time
from bmcs_expsim.utils.mlab_decorators import decorate_figure
from mayavi import mlab
import numpy as np
np.seterr(divide='ignore', invalid='ignore') 
import warnings
import matplotlib.pylab as plt
import numpy as np
from ibvpy.api import TStepBC, TFCyclicNonsymmetricConstant, TFBilinear
from ibvpy.bcond import BCSliceI, BCDof
from ibvpy.xmodel.xdomain_fe_grid_axisym import XDomainFEGridAxiSym
from ibvpy.fets import FETS2D4Q
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic
from ibvpy.tmodel.mats2D.mats2D_elastic.vmats2D_elastic import \
    MATS2DElastic

## FE Axisymmetric discretization

In [93]:
n_x_e = 20
n_inner_y_e = 10
n_outer_y_e = 10
L_x = 100.0 # [mm]
R_in = 25 # [mm]
dR = 10 # [mm]
R_out = 50 # [mm]
xd_inner = XDomainFEGridAxiSym(integ_factor = 2 * np.pi,
                               coord_min=(0, 0),
                               coord_max=(L_x, R_in),
                               shape=(n_x_e, n_inner_y_e),
                               fets=FETS2D4Q())
xd_middle = XDomainFEGridAxiSym(integ_factor = 2 * np.pi,
                               coord_min=(0, R_in),
                               coord_max=(L_x, R_in+dR),
                               shape=(n_x_e, 1),
                               fets=FETS2D4Q())
xd_outer = XDomainFEGridAxiSym(integ_factor = 2 * np.pi,
                               coord_min=(0, R_in+dR),
                               coord_max=(L_x, R_out),
                               shape=(n_x_e, n_outer_y_e),
                               fets=FETS2D4Q())
m_elastic = MATS3DElastic(E=37000, nu=0.18)
m_inelastic = MATS3DElastic(E=37000, nu=0.18)
m = TStepBC(
    domains=[(xd_inner, m_elastic),
             (xd_outer, m_elastic),
             (xd_middle, m_inelastic),
             ]
)

## Boundary and transition conditions

### Kinematic links between the domains

In [95]:
link_inner_middle = BCSliceI(var='u', dims=[0, 1],
                             slice=xd_inner.mesh.I[:, -1], 
                             link_slice=xd_middle.mesh.I[:, 0],
                             link_coeffs=[1.0, 1.0],
                             link_dims=[0, 1],
                             value=0.0
                            )
link_middle_outer = BCSliceI(slice=xd_outer.mesh.I[:, 0], 
                             var='u', dims=[0, 1],
                             link_slice=xd_middle.mesh.I[:, -1],
                             link_coeffs=[1, 1],
                             link_dims=[0, 1],
                             value=0,
                            )

### Compression/Traction fc

In [96]:
inner_fixed_1 = BCSliceI(slice=xd_inner.mesh.I[:, 0], var='u', dims=[1], value=0)
outer_fixed_0 = BCSliceI(slice=xd_outer.mesh.I[0, :], var='u', dims=[0], value=0)

outer_compression_slice = BCSliceI(slice=xd_outer.mesh.I[:, -1],
                                   var='u', dims=[1], value=0.01)

compression_dofs = outer_compression_slice.dofs
compression = -10000*L_x / len(compression_dofs)
outer_compression_force_first = [BCDof(var='f', dof=dof, value=compression) 
                                 for dof in compression_dofs ]
outer_compression_force_first[0].value *=0.5
outer_compression_force_first[-1].value *=0.5
bc1 = [inner_fixed_1, 
       outer_fixed_0,
       link_inner_middle,
       link_middle_outer
      ] + outer_compression_force_first

### Sliding dc

In [97]:
#lower_fixed_0 = BCSlice(slice=xd_lower.mesh[:, 0, :, 0], var='u', dims=[1], value=0)
#upper_fixed_1 = BCSlice(slice=xd_upper.mesh[0, :, 0, :], var='u', dims=[0], value=0)
#
#lower_slide = BCSlice(slice=xd_lower.mesh[0, :, 0, :], var='u', dims=[0], value=6)
#
#
#bc1 =   [lower_fixed_0, upper_fixed_1, lower_slide]

### Sliding fc

In [98]:
#lower_fixed_0 = BCSlice(slice=xd_lower.mesh[:, 0, :, 0], var='u', dims=[1], value=0)
#upper_fixed_1 = BCSlice(slice=xd_upper.mesh[0, :, 0, :], var='u', dims=[0], value=0)
#
#
#
#lower_slide = BCSlice(slice=xd_lower.mesh[0, :, 0, :], var='u', dims=[0], value=0)
#Force = 10 / len(lower_slide.dofs)
#lower_slide_force = [BCDof(var='f', dof=dof, value = Force) 
#             for dof in lower_slide.dofs]
#
#bc1 =   [lower_fixed_0, upper_fixed_1] + lower_slide_force

In [100]:
m.bc=bc1
m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 0.1
# Accessing the following property - ensures serialization
s.tstep.fe_domain.serialized_subdomains

In [101]:
s.reset()
s.run()

t:	 0.00(0), 
	 0.10(1), 
	 0.20(1), 
	 0.30(1), 
	 0.40(1), 
	 0.50(1), 
	 0.60(1), 
	 0.70(1), 
	 0.80(1), 
	 0.90(1), 
	 1.00(1), 



## Postprocesing

### Access the last displacement step

In [102]:
U_last = m.hist.U_t[-1]

In [103]:
U_last[xd_middle.o_Ia], U_last[xd_inner.o_Ia]

(array([[ 5.34240620e-17, -1.67962746e-02],
        [ 4.16875735e-17, -2.35147844e-02],
        [ 1.47479484e-03, -1.67962746e-02],
        [ 1.47479484e-03, -2.35147844e-02],
        [ 2.94958968e-03, -1.67962746e-02],
        [ 2.94958968e-03, -2.35147844e-02],
        [ 4.42438452e-03, -1.67962746e-02],
        [ 4.42438452e-03, -2.35147844e-02],
        [ 5.89917936e-03, -1.67962746e-02],
        [ 5.89917936e-03, -2.35147844e-02],
        [ 7.37397420e-03, -1.67962746e-02],
        [ 7.37397420e-03, -2.35147844e-02],
        [ 8.84876904e-03, -1.67962746e-02],
        [ 8.84876904e-03, -2.35147844e-02],
        [ 1.03235639e-02, -1.67962746e-02],
        [ 1.03235639e-02, -2.35147844e-02],
        [ 1.17983587e-02, -1.67962746e-02],
        [ 1.17983587e-02, -2.35147844e-02],
        [ 1.32731536e-02, -1.67962746e-02],
        [ 1.32731536e-02, -2.35147844e-02],
        [ 1.47479484e-02, -1.67962746e-02],
        [ 1.47479484e-02, -2.35147844e-02],
        [ 1.62227432e-02, -1.679

In [104]:
xd_middle.map_U_to_field(U_last)

array([[[[ 2.94958968e-04, -5.42101086e-19,  0.00000000e+00],
         [-5.42101086e-19, -6.71850982e-04,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00, -6.71850982e-04]],

        [[ 2.94958968e-04, -6.50521303e-19,  0.00000000e+00],
         [-6.50521303e-19, -6.71850982e-04,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00, -6.71850982e-04]],

        [[ 2.94958968e-04, -1.51788304e-18,  0.00000000e+00],
         [-1.51788304e-18, -6.71850982e-04,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00, -6.71850982e-04]],

        [[ 2.94958968e-04, -1.30104261e-18,  0.00000000e+00],
         [-1.30104261e-18, -6.71850982e-04,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00, -6.71850982e-04]]],


       [[[ 2.94958968e-04, -1.30104261e-18,  0.00000000e+00],
         [-1.30104261e-18, -6.71850982e-04,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00, -6.71850982e-04]],

        [[ 2.94958968e-04, -1.30104261e-18,  0.00000000e+0

In [88]:
if True:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'stress'
    strain_viz = Viz3DTensorField(vis3d=m.hist['stress'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()